In [9]:
# initialization
import numpy as np
from random import random, randint

# importing Qiskit
from qiskit import IBMQ, BasicAer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, execute

# import basic plot tools
from qiskit.visualization import plot_histogram

#CHANGE THESE
probability = 0.1
n = 6
func_type = 'balanced' #Can be wither 'balanced' or 'constant'

# set the length of the n-bit input string. 
def depolarizing_sim(circuit, n):
    if(probability >= random()):
        gate = randint(0,2)
        if gate == 0:
            circuit.x(n)
        elif gate == 1:
            circuit.y(n)
        else:
            circuit.z(n)

def dj_oracle(case, n):
    # We need to make a QuantumCircuit object to return
    # This circuit has n+1 qubits: the size of the input,
    # plus one output qubit
    oracle_qc = QuantumCircuit(n+1)
    
    # First, let's deal with the case in which oracle is balanced
    if case == "balanced":
        # First generate a random number that tells us which CNOTs to
        # wrap in X-gates:
        b = np.random.randint(1,2**n)
        # Next, format 'b' as a binary string of length 'n', padded with zeros:
        b_str = format(b, '0'+str(n)+'b')
        # Next, we place the first X-gates. Each digit in our binary string 
        # corresponds to a qubit, if the digit is 0, we do nothing, if it's 1
        # we apply an X-gate to that qubit:
        for qubit in range(len(b_str)):
            if b_str[qubit] == '1':
                oracle_qc.x(qubit)
        # Do the controlled-NOT gates for each qubit, using the output qubit 
        # as the target:
        for qubit in range(n):
            oracle_qc.cx(qubit, n)
        # Next, place the final X-gates
        for qubit in range(len(b_str)):
            if b_str[qubit] == '1':
                oracle_qc.x(qubit)

    # Case in which oracle is constant
    if case == "constant":
        # First decide what the fixed output of the oracle will be
        # (either always 0 or always 1)
        output = np.random.randint(2)
        if output == 1:
            oracle_qc.x(n)
    
    oracle_gate = oracle_qc.to_gate()
    oracle_gate.name = "Oracle" # To show when we display the circuit
    return oracle_gate

def dj_algorithm(oracle, n):
    dj_circuit = QuantumCircuit(n+1, n)
    # Set up the output qubit:
    dj_circuit.x(n)
    depolarizing_sim(dj_circuit,n)
    dj_circuit.h(n)
    depolarizing_sim(dj_circuit,n)
    # And set up the input register:
    for qubit in range(n):
        dj_circuit.h(qubit)
        depolarizing_sim(dj_circuit,qubit)
    # Let's append the oracle gate to our circuit:
    dj_circuit.append(oracle, range(n+1))
    for qubit in range(n):
        depolarizing_sim(dj_circuit,qubit)
    # Finally, perform the H-gates again and measure:
    for qubit in range(n):
        dj_circuit.h(qubit)
        depolarizing_sim(dj_circuit,qubit)
    
    for i in range(n):
        dj_circuit.measure(i, i)
    
    return dj_circuit

def dj_trial(trial):
    oracle_gate = dj_oracle(func_type, n)
    dj_circuit = dj_algorithm(oracle_gate, n)
    dj_circuit.draw(output="mpl")
    
    # use local simulator
    backend = BasicAer.get_backend('qasm_simulator')
    shots = 1024
    job = execute(dj_circuit, backend=backend, shots=shots)
    hist = job.result().get_counts()
    file = open(str(trial)+".txt","w")
    file.write(str(hist)+"\n")
    file.write(str(max(hist, key=hist.get)))
    file.close()
    

In [10]:
for trial in range(50):
    dj_trial(trial)